In [1]:
%pip install geopy ortools folium pulp

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import time
from datetime import timedelta
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import folium
import os
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpInteger, value, LpStatusOptimal, PULP_CBC_CMD

def safe_name(s):
    return str(s).replace(" ", "_").replace("-", "_").replace(".", "_").replace("/", "_").replace(",", "_")

folder_name = "Maps"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    

stores = pd.read_csv("stores_updated.csv")

forecasts = pd.read_csv("demand_forecasting.csv")
forecasts['date'] = pd.to_datetime(forecasts['date'])

oil_prices = pd.read_csv("oil_updated.csv")
oil_prices['date'] = pd.to_datetime(oil_prices['date'])

geolocator = Nominatim(user_agent="store_locator")

def get_city_coords(city, state, country="Ecuador"):
    try:
        location = geolocator.geocode(f"{city}, {state}, {country}")
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except Exception as e:
        print(f"Error geocoding {city}, {state}: {e}")
        return (None, None)

# Geocode unique city/state pairs

geolocator = Nominatim(user_agent="store_locator")

coords_dict = {}
for idx, row in  stores[["City", "State"]].drop_duplicates().iterrows():
    city, state = row["City"], row["State"]
    lat, lon = get_city_coords(city, state)
    if lat and lon:
        coords_dict[(city, state)] = (lat, lon)
    else:
        coords_dict[(city, state)] = (0, 0)  # fallback
    time.sleep(1)  # respect rate limits

# Since multiple stores in same city/state and don't have long/lat, 
# changing values a bit to distinguish different stores
def add_jitter(lat, lon, max_jitter=0.005):
    jitter_lat = np.random.uniform(-max_jitter, max_jitter)
    jitter_lon = np.random.uniform(-max_jitter, max_jitter)
    return lat + jitter_lat, lon + jitter_lon

stores["Latitude"] = None
stores["Longitude"] = None

for idx, row in stores.iterrows():
    base_lat, base_lon = coords_dict.get((row["City"], row["State"]), (0, 0))
    lat_j, lon_j = add_jitter(base_lat, base_lon)
    stores.at[idx, "Latitude"] = lat_j
    stores.at[idx, "Longitude"] = lon_j

# Warehouse location (HQ)
warehouse_location = get_city_coords(city='Amaguaña', state='Quito')
    

def assign_delivery_days(Is_perishable):
    if is_perishable:
        return list(pd.date_range("2017-08-16", "2017-08-31", freq="2D"))
    else:
        return list(pd.date_range("2017-08-16", "2017-08-31", freq="5D"))

delivery_plan = []
for (store_id, product), group in forecasts.groupby(["Store", "ProductFamily"]):
    is_perishable = group["IsPerishable"].iloc[0]
    delivery_dates = assign_delivery_days(is_perishable)
    for date in delivery_dates:
        period_end = date + timedelta(days=1 if is_perishable else 4)
        mask = (group["date"] >= date) & (group["date"] <= period_end)
        demand_sum = np.ceil(group.loc[mask, "Demand"].sum())
        if demand_sum > 0:
            delivery_plan.append({
                "Store": store_id,
                "ProductFamily": product,
                "DeliveryDate": date,
                "Demand": demand_sum
            })
delivery_schedule_df = pd.DataFrame(delivery_plan)

# Merge stores info to delivery schedule for location
delivery_schedule_df = delivery_schedule_df.merge(stores, on="Store")

In [3]:
# ---- MILP Product Distribution & Stock Optimization ----

fee_per_unit = 0.0005
vehicle_capacity = 30000  

grouped = delivery_schedule_df.groupby(['DeliveryDate', 'ProductFamily'])
milp_results = []

for (delivery_date, product_family), group in grouped:
    print(f"\nMILP for {product_family} on {delivery_date.date()}")

    stores_group = group.reset_index(drop=True)
    store_ids = stores_group["Store"].tolist()
    demands = stores_group["Demand"].tolist()
    locations = list(zip(stores_group["Latitude"], stores_group["Longitude"]))

    dist_warehouse_to_stores = [great_circle(warehouse_location, loc).kilometers for loc in locations]

    prob = LpProblem(f"ProductDistribution_{safe_name(product_family)}_{safe_name(delivery_date.date())}", LpMinimize)

    qty_vars = {store_id: LpVariable(f"Qty_{store_id}", lowBound=0, cat=LpInteger) for store_id in store_ids}

    prob += lpSum([qty_vars[store_id] * dist * fee_per_unit for store_id, dist in zip(store_ids, dist_warehouse_to_stores)])

    for store_id, demand in zip(store_ids, demands):
        prob += qty_vars[store_id] >= demand, f"Demand_{store_id}"

    total_demand = sum(demands)
    vehicles_needed = int(np.ceil(total_demand / vehicle_capacity))
    prob += lpSum([qty_vars[store_id] for store_id in store_ids]) <= vehicles_needed * vehicle_capacity, "VehicleCapacity"

    solver = PULP_CBC_CMD(msg=False)
    prob.solve(solver)

    if prob.status != LpStatusOptimal:
        print("No optimal solution found!")
        continue

    shipment_quantities = {store_id: int(qty_vars[store_id].varValue) for store_id in store_ids}

    print(f"Vehicles needed (MILP): {vehicles_needed}")
    print("Shipments:")
    for sid, qty in shipment_quantities.items():
        print(f" Store {sid}: {qty} units")

    milp_results.append({
        "DeliveryDate": delivery_date,
        "ProductFamily": product_family,
        "Stores": stores_group,
        "ShipmentQuantities": shipment_quantities,
        "VehiclesNeeded": vehicles_needed
    })

shipment_aggregated = {}

for res in milp_results:
    delivery_date = res["DeliveryDate"]
    shipment_quantities = res["ShipmentQuantities"]

    for store_id, qty in shipment_quantities.items():
        key = (delivery_date, store_id)
        shipment_aggregated[key] = shipment_aggregated.get(key, 0) + qty

violations = []

for (delivery_date, store_id), total_qty in shipment_aggregated.items():
    store_capacity = stores.loc[stores["Store"] == store_id, "StoreCapacity"].values
    if len(store_capacity) == 0:
        print(f"Warning: Store {store_id} capacity not found!")
        continue
    capacity = store_capacity[0]

    if total_qty > capacity:
        violations.append({
            "DeliveryDate": delivery_date,
            "Store": store_id,
            "TotalShipment": total_qty,
            "Capacity": capacity,
            "Excess": total_qty - capacity
        })

if violations:
    print("\n*** Store capacity violations detected! Removing those stores for the affected delivery dates ***")
    for v in violations:
        print(f"Date: {v['DeliveryDate'].date()}, Store: {v['Store']}, Shipment: {v['TotalShipment']}, Capacity: {v['Capacity']}, Excess: {v['Excess']}")

    filtered_results = []
    for res in milp_results:
        delivery_date = res["DeliveryDate"]
        product_family = res["ProductFamily"]
        stores_group = res["Stores"]
        shipment_quantities = res["ShipmentQuantities"]
        vehicles_needed = res["VehiclesNeeded"]

        violating_stores = {v['Store'] for v in violations if v['DeliveryDate'] == delivery_date}

        mask = ~stores_group["Store"].isin(violating_stores)
        filtered_stores = stores_group[mask].reset_index(drop=True)

        filtered_shipment_quantities = {sid: qty for sid, qty in shipment_quantities.items() if sid not in violating_stores}

        total_qty = sum(filtered_shipment_quantities.values())
        vehicles_needed = int(np.ceil(total_qty / vehicle_capacity)) if total_qty > 0 else 0

        if len(filtered_stores) > 0 and vehicles_needed > 0:
            filtered_results.append({
                "DeliveryDate": delivery_date,
                "ProductFamily": product_family,
                "Stores": filtered_stores,
                "ShipmentQuantities": filtered_shipment_quantities,
                "VehiclesNeeded": vehicles_needed
            })
        else:
            print(f"All shipments removed for {product_family} on {delivery_date.date()} due to capacity violation")

    milp_results = filtered_results
else:
    print("\nNo store capacity violations detected.")


MILP for AUTOMOTIVE on 2017-08-16
Vehicles needed (MILP): 4
Shipments:
 Store 1: 53 units
 Store 2: 50 units
 Store 3: 56 units
 Store 4: 11140 units
 Store 5: 43 units
 Store 6: 1696 units
 Store 7: 52 units
 Store 8: 3090 units
 Store 9: 3232 units
 Store 10: 811 units
 Store 11: 712 units
 Store 12: 686 units
 Store 13: 11875 units
 Store 14: 61 units
 Store 15: 46 units
 Store 16: 68 units
 Store 17: 83 units
 Store 18: 45 units
 Store 19: 939 units
 Store 20: 57 units
 Store 21: 75 units
 Store 22: 53 units
 Store 23: 562 units
 Store 24: 51 units
 Store 25: 1820 units
 Store 26: 891 units
 Store 27: 49 units
 Store 28: 56 units
 Store 29: 1649 units
 Store 30: 315 units
 Store 31: 13547 units
 Store 32: 50 units
 Store 33: 76 units
 Store 34: 43 units
 Store 35: 41 units
 Store 36: 50 units
 Store 37: 15156 units
 Store 38: 43 units
 Store 39: 3320 units
 Store 40: 56 units
 Store 41: 4429 units
 Store 42: 4169 units
 Store 43: 1332 units
 Store 44: 926 units
 Store 45: 629 unit

In [4]:
# ---- VRP Warehouse to Branch Transportation ----
def split_large_demands(stores, shipment_quantities, vehicle_capacity):
    new_stores = []
    new_demands = []
    for _, store in stores.iterrows():
        store_id = store["Store"]
        demand = shipment_quantities[store_id]
        num_splits = int(np.ceil(demand / vehicle_capacity))
        split_demand = demand / num_splits
        for split_i in range(num_splits):
            split_store = store.copy()
            split_store["SplitID"] = f"{store_id}_{split_i}"
            new_stores.append(split_store)
            new_demands.append(split_demand)
    new_stores_df = pd.DataFrame(new_stores).reset_index(drop=True)
    return new_stores_df, new_demands

def create_distance_matrix(locations):
    size = len(locations)
    dist_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            dist_matrix[i][j] = great_circle(locations[i], locations[j]).kilometers
    return dist_matrix

def solve_vrp(distance_matrix, demands, vehicle_capacities, depot=0):
    manager = pywrapcp.RoutingIndexManager(len(distance_matrix), len(vehicle_capacities), depot)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_idx, to_idx):
        return int(distance_matrix[manager.IndexToNode(from_idx)][manager.IndexToNode(to_idx)] * 1000)

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def demand_callback(from_idx):
        return int(demands[manager.IndexToNode(from_idx)])

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, vehicle_capacities, True, "Capacity")

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.time_limit.seconds = 30
    search_parameters.log_search = False

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        routes = []
        for vehicle_id in range(len(vehicle_capacities)):
            index = routing.Start(vehicle_id)
            route = []
            while not routing.IsEnd(index):
                node = manager.IndexToNode(index)
                route.append(node)
                index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index)) 
            routes.append(route)
        return routes
    else:
        return None

for res in milp_results:
    delivery_date = res["DeliveryDate"]
    product_family = res["ProductFamily"]
    stores = res["Stores"]
    shipment_quantities = res["ShipmentQuantities"]
    vehicles_needed = res["VehiclesNeeded"]

    print(f"\nVRP routing for {product_family} on {delivery_date.date()}, Vehicles: {vehicles_needed}")

    filtered_stores = stores[[shipment_quantities[sid] > 0 for sid in stores["Store"]]].reset_index(drop=True)

    filtered_stores, filtered_qty = split_large_demands(filtered_stores, shipment_quantities, vehicle_capacity)

    locations = [warehouse_location] + list(zip(filtered_stores["Latitude"], filtered_stores["Longitude"]))
    demands = [0] + filtered_qty  
    vehicle_capacities = [vehicle_capacity] * vehicles_needed

    dist_matrix = create_distance_matrix(locations)
    routes = solve_vrp(dist_matrix, demands, vehicle_capacities, depot=0)

    if routes is None:
        print("No VRP solution found.")
        continue

    for v, route in enumerate(routes):
        route_store_ids = []
        for i in route:
            if i == 0:
                route_store_ids.append("Depot")
            else:
                route_store_ids.append(filtered_stores.loc[i - 1, "SplitID"])
        print(f" Vehicle {v + 1} route: {route_store_ids}")

    total_route_distance = 0
    for route in routes:
        route_distance = 0
        for i in range(len(route) - 1):
            from_node = route[i]
            to_node = route[i+1]
            route_distance += dist_matrix[from_node][to_node]
        total_route_distance += route_distance

    # Oil price on that date, convert barrel price to liter price (1 barrel = ~159 liters)
    oil_price = oil_prices[oil_prices["date"].dt.date == delivery_date.date()]["OilPrice"].values[0] / 159

    total_cost = total_route_distance * oil_price + fee_per_unit * total_demand

    print(f"Total distance traveled by all vehicles: {total_route_distance:.2f} km")
    print(f"Oil price on {delivery_date.date()}: {oil_price} USD/liter")
    print(f"Total delivery cost for {product_family} on {delivery_date.date()}: {total_cost:.2f} USD")

    m = folium.Map(location=warehouse_location, zoom_start=7)
    folium.Marker(location=warehouse_location, tooltip="Warehouse HQ", icon=folium.Icon(color='red')).add_to(m)

    colors = ['blue', 'green', 'purple', 'orange', 'darkred']
    for v, route in enumerate(routes):
        points = [locations[i] for i in route]
        folium.PolyLine(points, color=colors[v % len(colors)], weight=5, opacity=0.8,
                        tooltip=f"Vehicle {v + 1}").add_to(m)
        for idx in route[1:-1]:
            store = filtered_stores.loc[idx - 1]
            folium.Marker(
                location=(store["Latitude"], store["Longitude"]),
                tooltip=f"Store {store['Store']} Demand: {filtered_qty[idx - 1]:.1f}",
                icon=folium.Icon(color=colors[v % len(colors)], icon='truck', prefix='fa')
            ).add_to(m)

    map_filename = f"{safe_name(product_family)}_Distribution_Transportation_Plan_{delivery_date.date()}.html"
    m.save(f"{folder_name}/{map_filename}")

    print(f"Map saved as {map_filename}")


VRP routing for AUTOMOTIVE on 2017-08-16, Vehicles: 4
 Vehicle 1 route: ['Depot', '6_0', '10_0', '4_0', '1_0', '8_0', '7_0', '47_0', '18_0', '2_0', '49_0', '3_0', '48_0', '17_0', '13_0', 'Depot']
 Vehicle 2 route: ['Depot', '22_0', '14_0', '37_0', '42_0', '39_0', '38_0', '40_0', '41_0', '19_0', '23_0', '50_0', '12_0', 'Depot']
 Vehicle 3 route: ['Depot', '11_0', '15_0', '43_0', '52_0', '53_0', '25_0', '35_0', '32_0', '28_0', '51_0', '29_0', '26_0', '24_0', '30_0', '34_0', '27_0', '36_0', '31_0', '33_0', '54_0', '5_0', '16_0', '21_0', 'Depot']
 Vehicle 4 route: ['Depot', '20_0', '45_0', '9_0', '44_0', '46_0', 'Depot']
Total distance traveled by all vehicles: 2363.22 km
Oil price on 2017-08-16: 0.2943396226415094 USD/liter
Total delivery cost for AUTOMOTIVE on 2017-08-16: 710.46 USD
Map saved as AUTOMOTIVE_Distribution_Transportation_Plan_2017-08-16.html

VRP routing for BABY CARE on 2017-08-16, Vehicles: 8
No VRP solution found.

VRP routing for BEAUTY on 2017-08-16, Vehicles: 3
 Vehic